# Code Summarization with Deep Reinforcement Learning
Paper[1] is linked: [Improving Code Summarization with Deep Reinforcement Learning](https://arxiv.org/abs/1811.07234)  
We're attempting to add concepts from Paper[2] [Unsupervised Translation of Programming Languages](https://arxiv.org/pdf/2006.03511)  
How? Not sure...  
I'm thinking of using their masking technique to finetune the product of Paper[1]. Essentially, with a given

# 1. Load Data

In [ ]:
import pandas as pd
import os
cache_dir = 'F:\\.cache\\huggingface\\'
os.environ['HF_HOME'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = cache_dir
import code_search_net
import datasets

In [2]:
# You can use the below line instead of you don't want to load locally.
# BUt I havne't figured out to only choose certain datasets, I think its the uploader's fault
# Total is like 10+GB's
# d = datasets.load_dataset('code-search-net/code_search_net')

In [4]:
# What language you used for the data, otherwise default to all
dl_manager = datasets.DownloadManager('python')
csn = code_search_net.CodeSearchNet(config_name='python')
# You can remove the loc variable. My c:drive has no space so I did this instead.
generators = csn._split_generators(dl_manager, loc = 'F:\\.cache\\huggingface\\')

Repo card metadata block was not found. Setting CardData to empty.


In [5]:
raw_data = []
for i, data in csn._generate_examples(generators[0].gen_kwargs['filepaths'][:1]):
    raw_data.append(data)

In [ ]:
# Of interest:
# func_code_tokens
# func_documentation_tokens
# possibly func_path_in_repository?
print(len(raw_data))
raw_data[0].keys()


30000


dict_keys(['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'])

# 2. Create Inputs
"To convert code into sequential text, we tokenize the code by `{. , ” ’ : ; ) ( ! (space)}`, which  has been used in [8].  
We tokenize the comment by {(space)}" - Paper

## 2a. Vocab Creation
- Reparse func_code_string into tokens
- Might respase func_documaetnation_string
- lower case

In [ ]:
vocab_code = {'SOS' : 0} # Is this necessary?
max_code = 0
i = 1
vocab_comm = {'SOS' : 0}
max_comm = 0
j = 1
for f in raw_data:
    # I think we will need to preprocess out own toeksn btu for right now prrof of concenpt:
    for code_token in f['func_code_tokens']:
        if code_token not in vocab_code.keys():
            vocab_code[code_token] = i
            i += 1
    max_code = max(max_code, len(f['func_code_tokens']))
    for comm_token in f['func_documentation_tokens']:
        if comm_token not in vocab_comm.keys():
            vocab_comm[comm_token] = j
            j += 1
    max_comm = max(max_comm, len(f['func_documentation_tokens']))
print('Vocab / Max Sentence Length')
print(len(vocab_code), max_code, ' | ', len(vocab_comm), max_comm)


Vocab / Max Sentence Length
202545 4091  |  21314 564


In [26]:
# You can see some of these ar wrong which means we'll have to probably preprocess it outselves.
vocab_code

{'SOS': 0,
 'def': 1,
 'train': 2,
 '(': 3,
 'train_dir': 4,
 ',': 5,
 'model_save_path': 6,
 '=': 7,
 'None': 8,
 'n_neighbors': 9,
 'knn_algo': 10,
 "'ball_tree'": 11,
 'verbose': 12,
 'False': 13,
 ')': 14,
 ':': 15,
 'X': 16,
 '[': 17,
 ']': 18,
 'y': 19,
 '# Loop through each person in the training set': 20,
 'for': 21,
 'class_dir': 22,
 'in': 23,
 'os': 24,
 '.': 25,
 'listdir': 26,
 'if': 27,
 'not': 28,
 'path': 29,
 'isdir': 30,
 'join': 31,
 'continue': 32,
 '# Loop through each training image for the current person': 33,
 'img_path': 34,
 'image_files_in_folder': 35,
 'image': 36,
 'face_recognition': 37,
 'load_image_file': 38,
 'face_bounding_boxes': 39,
 'face_locations': 40,
 'len': 41,
 '!=': 42,
 '1': 43,
 '# If there are no people (or too many people) in a training image, skip the image.': 44,
 'print': 45,
 '"Image {} not suitable for training: {}"': 46,
 'format': 47,
 '"Didn\'t find a face"': 48,
 '<': 49,
 'else': 50,
 '"Found more than one face"': 51,
 '# Add fa

In [28]:
vocab_comm

{'SOS': 0,
 'Trains': 1,
 'a': 2,
 'k': 3,
 '-': 4,
 'nearest': 5,
 'neighbors': 6,
 'classifier': 7,
 'for': 8,
 'face': 9,
 'recognition': 10,
 '.': 11,
 'Recognizes': 12,
 'faces': 13,
 'in': 14,
 'given': 15,
 'image': 16,
 'using': 17,
 'trained': 18,
 'KNN': 19,
 'Shows': 20,
 'the': 21,
 'results': 22,
 'visually': 23,
 'Convert': 24,
 'dlib': 25,
 'rect': 26,
 'object': 27,
 'to': 28,
 'plain': 29,
 'tuple': 30,
 '(': 31,
 'top': 32,
 'right': 33,
 'bottom': 34,
 'left': 35,
 ')': 36,
 'order': 37,
 'Make': 38,
 'sure': 39,
 'is': 40,
 'within': 41,
 'bounds': 42,
 'of': 43,
 'Given': 44,
 'list': 45,
 'encodings': 46,
 'compare': 47,
 'them': 48,
 'known': 49,
 'encoding': 50,
 'and': 51,
 'get': 52,
 'euclidean': 53,
 'distance': 54,
 'each': 55,
 'comparison': 56,
 'The': 57,
 'tells': 58,
 'you': 59,
 'how': 60,
 'similar': 61,
 'are': 62,
 'Loads': 63,
 'an': 64,
 'file': 65,
 'jpg': 66,
 'png': 67,
 'etc': 68,
 'into': 69,
 'numpy': 70,
 'array': 71,
 'Returns': 72,
 'bou

## 2b. Create AST
ASTs to binary trees by the following two steps which have been adopted in [28]:  
- a) Split nodes with more than 2 children, generate a new right child together with the old left child as its children, and then put all children except the leftmost as the children of this new node.   
Repeat this operation in a top-down way until only nodes with 0, 1, 2 children left;
- b) Combine nodes with 1 child with its child.  

(from the paper)

In [ ]:
import ast
n = ast.parse(raw_data[0]['func_code_string'])
print("children: " + str([x for x in ast.iter_child_nodes(n)]) + "\\n")


children: [<ast.FunctionDef object at 0x000002123F727FA0>]\n


# 2c. Create Sequential Input


## 2d. Pack them into a data structure
We should form Train/Test Splits, I think the `raw_data` variable already holds this. Should be something of:  
```
inputs[0] = {  
'seq' : list of sequential representation of code (converted to ID from sectiion 2a, lists from 2c)
'ast' : list of ast representation of code (converted to ID from section 2a, AST from 2b.)
}
make sure we padding them, and have <SOS> <EOS>
```

# 3. Model Definition
Probably hardest bit, I got as many variables from the paper as I could.  
i think a lot of it will be similar to what we did in class projects (Lab 5)

## 3a. Model params

In [ ]:
# Taken from paper
batch_size = 64
hidden_size = 512
embedding_size = 512 #LSTM, Layers
lr = 0.001
skip_num = 0
code_vocab_size = len(vocab_code)
#code_max_len = max_code
comment_vocab_size = len(vocab_comm)
#comment_max_len = max_comm # This and max_c

## 3b. Actor Network

## 3c. Critic Network

## 3d. Training Loop?

# 4. Modifications 

# Solver test